In [ ]:
import pandas as pd

In [ ]:
billionaire_df = pd.read_csv("../data/billionaire.csv")
billionaire_df.head()

In [ ]:
coord_df = pd.read_csv("../data/country_lat_lon.csv")
coord_df.head()

In [ ]:
merged_df = pd.merge(billionaire_df, coord_df, left_on="Country", right_on="country")
merged_df.head()

In [ ]:
final_df = merged_df[["Name", "NetWorth", "Country", "Source", "Rank", "latitude", "longitude"]]
final_df.head()

In [ ]:
final_df.to_csv("../data/merged_data.csv")